### Load libraries and variables

In [ ]:
from mace.calculators import MACECalculator
from ase.build.surface import mx2
import numpy as np
import os
from ase.vibrations import Vibrations
from ase.vibrations import VibrationsData
import matplotlib.pyplot as plt
%matplotlib inline
from ase.optimize import BFGS
from scipy.interpolate import interp2d
import ase.units
from ase.io import read, Trajectory


calc = MACECalculator(model_paths="../checkpoints/MoWSSe_MACE_new_version_run-123.model", device="cuda", default_dtype="float64")
foldername = 'raman_data_converged_new_version'

a_optB88_QE = {}
a_optB88_QE[1,0] = 3.190902132
a_optB88_QE[0,0] = 3.186305534
a_optB88_QE[1,1] = 3.322874304
a_optB88_QE[0,1] = 3.322339468

t_optB88_QE = {}
t_optB88_QE[1,0] = 3.1570497014
t_optB88_QE[0,0] = 3.140170013
t_optB88_QE[1,1] = 3.367178811
t_optB88_QE[0,1] = 3.3492559072

# Create linear interpolation
xd = np.array([0,1,0,1])
yd = np.array([0,0,1,1])
ad = [0]*len(xd)
td = [0]*len(xd)
for q in range(len(xd)):
    ad[q] = a_optB88_QE[xd[q],yd[q]]
    td[q] = t_optB88_QE[xd[q],yd[q]]
a_xy = interp2d(xd,yd,ad)
t_xy = interp2d(xd,yd,td)


### Load methods

In [ ]:
def read_raman(filename):
    '''
    Read Raman tensors from ph.rec.out file in shape (3N,3,3)
    where N is number of atoms in primitive unit cell
    '''

    #Open file and read its lines
    file = open(filename)
    lines = file.readlines()

    i=0

    #Pass through lines to find keywords
    while i<len(lines):
        
        #Get number of atoms in unit cell
        if 'number of atoms/cell' in lines[i]:
            n_atoms = int(lines[i].split()[-1])
            raman_tensors = np.zeros((3*n_atoms,3,3))
        
        #Read Raman tensor
        if 'Raman tensor (A^2)' in lines[i]:
            i+=2
            for j in range(3*n_atoms):
                i+=1
                tensor = np.zeros((3,3))
                for k in range(3):

                    row = [float(x) for x in lines[i].split()]
                    tensor[k]=row
                    i+=1
                raman_tensors[j]=tensor
            file.close()
            return raman_tensors

        i+=1
    file.close()
    print('Incomplete data for Raman tensors')
    
def get_effective_raman(x,y):
    '''
    Get effective Raman tensor for (x,y) composition
    '''
    raman_mos2 = read_raman('mos2_converged.ph.rec.out')
    raman_mose2 = read_raman('mose2_converged.ph.rec.out')
    raman_ws2 = read_raman('ws2_converged.ph.rec.out')
    raman_wse2 = read_raman('wse2_converged.ph.rec.out')
    
    effective_tensor = (1-x)*(1-y)*raman_mos2 + x*(1-y)*raman_ws2 + (1-x)*y*raman_mose2 + x*y*raman_wse2
    
    return effective_tensor 

def read_eigenvectors(filename):
    '''
    Read eigenvectors for primitive cell from Espresso dynmat output files
    '''
    file = open(filename)
    lines = file.readlines()
    i=0
    mode_index = 0
    eigenvectors=np.zeros((9,3,3))

    while i<len(lines):

        if 'freq' in lines[i]:
            atom_num=0
            for j in range(3):

                i+=1

                float_list = lines[i].split()[1:-1] #No round brackets included
                eige = [float(num) for k,num in enumerate(float_list) if k%2==0]
                eigenvectors[mode_index,atom_num] = eige
                atom_num+=1
            mode_index+=1

        i+=1
    return eigenvectors

def passlog():
    pass

def get_eigenvectors_and_freq(atoms,calc):
    '''
    Get eigenvectors for alloy supercell 
    '''
    
    atoms.calc = calc
    dyn = BFGS(atoms)
    dyn.log = passlog
    dyn.run(fmax=0.001)

    vib = Vibrations(atoms)
    vib.clean()
    vib.run()
    vib_data = vib.get_vibrations()
    energies,sc_eige = vib_data.get_energies_and_modes()
    vib.clean()
    frequencies = energies/ase.units.invcm
    
    return sc_eige,frequencies
      
def get_sc_tensors(sup_eige,R_pc):
    '''
    Get raman tensors for alloy supercell
    using effective raman tensors obtained from supercells
    '''
    raman_tensors = np.zeros((sup_eige.shape[0],3,3))
    natp = int(R_pc.shape[0]/3)
    
    for nus in range(sup_eige.shape[0]):
        for iats in range(sup_eige.shape[1]):
            for lp in range(3):
                raman_tensors[nus,:,:] += sup_eige[nus,iats,lp] * R_pc[(iats % natp)*3+lp,:,:]
    
    return raman_tensors

def make_alloy(x,y,N):
    '''
    Make MoWSSe alloy having formula Mo(1-x)W(x)S(2-2y)Se(2y)
    '''
    prim_cell = mx2(formula='MoS2',kind='2H',a=a_xy(x,y),thickness=t_xy(x,y),vacuum=8,size=(1,1,1))
    atoms = prim_cell*(N,N,1)
    nflip_x = int(len(atoms)/3*x)
    flip_x = np.random.choice(range(0,len(atoms),3),nflip_x,replace=False)
    for f in flip_x:
        atoms.symbols[f] = 'W'

    #Flip S with Se
    nflip_y = int(2*len(atoms)/3*y)
    flip_y = np.random.choice([i for i in range(len(atoms)) if i%3!=0],nflip_y,
                              replace=False)
    for f in flip_y:
        atoms.symbols[f] = 'Se'
    return atoms

def get_raman_intensities(raman_tensors):
    '''
    Get Raman intensities for the tensors
    '''
    
    raman_intensities = np.zeros(raman_tensors.shape[0])
    
    for i in range(raman_tensors.shape[0]):
        
        tensor = raman_tensors[i]
        r_xx,r_yy,r_zz  = tensor.diagonal() ##diagonal terms
        r_xy,r_yz,r_xz = tensor[[0,1,2],[1,2,0]] ##off diagonal terms
        
        #Use formula 45a^2 + 7gamma^2
        a = (r_xx+r_yy+r_zz)/3
        gamma_sq = 0.5*((r_xx-r_yy)**2+(r_xx-r_zz)**2+(r_yy-r_zz)**2
                        +6*(r_xy**2 + r_yz**2 + r_xz**2))
        
        intensity = 45*a**2 + 7*gamma_sq
        
        raman_intensities[i] = intensity
        
    return raman_intensities   

def get_alloy_raman(x,y,N,calc):
    '''
    Get all Raman data of alloy supercell having
    composition (x,y) and size NxNx1
    '''
    sup = make_alloy(x,y,N)
    sup_eige,frequencies = get_eigenvectors_and_freq(sup,calc)
    masses = sup.get_masses()
    #sup_eige = sup_eige * masses[np.newaxis, :, np.newaxis]**-0.5
    #weff = get_effective_weights(sup_eige,x,y)/np.sqrt(N*N)
    Reff = get_effective_raman(x,y)
    alloy_tensors = get_sc_tensors(sup_eige,Reff)/np.sqrt(N*N)
    raman_intensities = get_raman_intensities(alloy_tensors)


    return sup,raman_intensities,frequencies,alloy_tensors

def gaussian(x,amplitude,mean,std):
    '''
    Gaussian curve A*e^(-(x-mean)^2/2*std^2) 
    '''
    return amplitude*np.exp(-(x-mean)**2/(2*std**2))

def get_spectra_curve(x,intensities,frequencies,std):
    '''
    Get Raman spectrum by summing up gaussians centered at 
    vibrational frequencies,and peak amplitude as the 
    corresponding intensities
    '''
    
    y = np.zeros(len(x))

    for i,intensity in enumerate(intensities):
        y = y + gaussian(x,intensity,frequencies[i],std)
    return x,y

def get_average_curve(x,y,N,configs,std,calc,trajname,foldername):
    '''
    Get the average Raman spectra over a particular number (nconfigs) 
    of random configurations of a given (x,y) alloy
    '''
    
    xspace = np.linspace(0,500,5000)
    curve = np.zeros(len(xspace))
    
    if not os.path.exists(foldername):
        os.makedirs(foldername)
    
    tout = Trajectory(f'{foldername}/{trajname}.traj',"w")
    
    for i in range(configs):
        
        print('disorder config ',i)
        atoms,intensities,frequencies,raman_tensors = get_alloy_raman(x,y,N,calc)
        tout.write(atoms)
        np.savez(f'{foldername}/{trajname}_config_{i}.npz',
                 intensities=intensities,freqs=frequencies,tensors=raman_tensors)
        
        curve = curve + get_spectra_curve(xspace,intensities,frequencies,std)[1]
    
    tout.close()
    
    curve = curve/configs
    
    return xspace,curve

### Get Raman data for ternary Mo$\bf _{1-x}$W$\bf _x$S$\bf _2$ alloys

In [ ]:
allfreqs_MoWS2 = []
allintens_MoWS2 = []
y=0
xs = [0.00,0.20,0.42,0.53,0.61,0.77,0.83,0.95,1.0]
for x in xs:
    nconfig = 1 if (x==0.0 or x==1.0) else 15
    print(x,y,nconfig)
    freqs,intens = get_average_curve(x,y,6,nconfig,4,calc,f'MoWS2_x{x:.3f}y{y:.3f}',foldername)
    allfreqs_MoWS2.append(freqs)
    allintens_MoWS2.append(intens)

### Get Raman data for ternary MoS$\bf _{2-2y}$Se$\bf _{2y}$ alloys

In [ ]:
allfreqs_MoSSe = []
allintens_MoSSe = []
ys = [0.00,0.28,0.33,0.47,0.52,0.67,0.83,0.95,1.0]

x = 0.0
for y in ys:
    nconfig = 1 if (y==0.0 or y==1.0) else 15
    print(x,y,nconfig)
    freqs,intens = get_average_curve(x,y,6,nconfig,4,calc,f'MoSSe_x{x:.3f}y{y:.3f}',foldername)
    allfreqs_MoSSe.append(freqs)
    allintens_MoSSe.append(intens)

### Get Raman data for ternary WS$\bf _{2-2y}$Se$\bf _{2y}$ alloys

In [ ]:
allfreqs_WSSe = []
allintens_WSSe = []
#ys = [0.00,0.30,0.5,0.7,1.0]
ys=[0.83,0.95]
x=1.00
for y in ys:
    nconfig = 1 if (y==0.0 or y==1.0) else 15
    print(y,nconfig)
    freqs,intens = get_average_curve(x,y,6,nconfig,4,calc,f'WSSe_x{x:.3f}y{y:.3f}',foldername)
    allfreqs_WSSe.append(freqs)
    allintens_WSSe.append(intens)

### Get Raman data for ternary Mo$\bf _{1-x}$W$\bf _x$Se$\bf _2$ alloys

In [ ]:
allfreqs_MoWSe2 = []
allintens_MoWSe2 = []
y=1
xs = [0.00,0.21,0.39,0.53,0.73,0.81,1.0]
for x in xs:
    nconfig = 1 if (x==0.0 or x==1.0) else 15
    print(x,y,nconfig)
    freqs,intens = get_average_curve(x,y,6,nconfig,4,calc,f'MoWSe2_x{x:.3f}y{y:.3f}',foldername)
    allfreqs_MoWSe2.append(freqs)
    allintens_MoWSe2.append(intens)

###  Get Raman data for quaternary Mo$\bf _{1-x}$W$\bf _x$S$\bf _{2-2y}$Se$\bf _{2y}$ alloys

In [ ]:
xs = [0.25,0.5,0.75]
ys = [0.25,0.5,0.75]

allfreqs_MoWSSe = []
allintens_MoWSSe = []

for x in xs:
    for y in ys:
        nconfig = 15
        print(x,y,nconfig)
        freqs,intens = get_average_curve(x,y,6,nconfig,4,calc,f'MoWSSe_x{x:.3f}y{y:.3f}',foldername)
        allfreqs_MoWSSe.append(freqs)
        allintens_MoWSSe.append(intens)